In [1]:
import matplotlib.pyplot as plt
from rlx import geo
import numpy as np
from rlx.utils import read_password_protected_zip, STANDARD_COLORS
#%matplotlib inline
import shapely as sh
from rlx.utils import pbar
import pandas as pd
from skimage.io import imread
import os
#API_key = read_password_protected_zip("google_apikey.zip")

In [2]:
d = pd.read_csv("data/shapefile_categories_map.csv", index_col="CONSTRU")
m = {i:v.values[0] for i,v in d.iterrows()}

In [3]:
print np.unique(d.SegConstru)

['Deporte-jardin' 'Edificio' 'Jardin' 'Piscina' 'Suelo']


In [4]:
d

,SegConstru
CONSTRU,
P,Suelo
P+I,Edificio
IVOL,Edificio
MAR,Edificio
TEN,Edificio
TRF,Edificio
-I+I+POR,Edificio
I,Edificio
-I+II,Edificio


# 1. create layer object

In [5]:
reload(geo)

shapefile_name = "/Users/rlx/datasets/google_maps/alcala_henares.catastro_urbano_sin_historia_28_05_UA_2018-01-20_SHF/CONSTRU.SHP"
L2 = geo.GoogleMaps_Shapefile_Layer("L2", shapefile_name, utm_zone_number=30, utm_zone_letter="T")

reading shapefile
converting to latlon
generating polygons


100% (96149 of 96149) |###################| Elapsed Time: 0:00:05 Time: 0:00:05


# 2. Get list of existing images

In [6]:
from rlx.utils import command

_, imgs_fnames, _ = command("find /Users/rlx/datasets/google_maps/alcala_henares_20180211 -name '*jpg'")
len(imgs_fnames)

14001

splits test/train in half by latitude

In [7]:
latlons = np.r_[[list(np.r_[i.split("_")][[-7,-6]].astype(float)) for i in imgs_fnames]]
median = np.median(latlons, axis=0)
imgs_train = np.r_[imgs_fnames][[latlons[:,0]>median[0]]]
imgs_test  = np.r_[imgs_fnames][[latlons[:,0]<=median[0]]]

In [8]:
len(imgs_train), len(imgs_test), len(np.unique(list(imgs_train)+list(imgs_test)))

(7000, 7001, 14001)

# 3. Define coloring function for each multipolygon in shapefile

In [9]:
def func_label(record, alpha=0):
    if record.CONSTRU not in m.keys():
        return "red", alpha
    c = m[record.CONSTRU]
    if c=="Edificio":
        return "red", alpha
    if c=="Suelo":
        return "yellow", alpha
    if c=="Piscina":
        return "blue", alpha
    if c=="Deporte-jardin" or c=="Jardin":
        return "green", alpha
    else:
        print "unkown", c
        return "black", alpha
    

# 4. Recover image object for each image and create corresponding layer

In [10]:
confs = [{ "color_func": lambda x: func_label(x, alpha=0), 
           "target_dir": "/Users/rlx/datasets/google_maps/alcala_henares_20180211_L2_imgs_train/",
           "overlay_original": True, "verbose": False},
         
         { "color_func": lambda x: func_label(x, alpha=1), 
           "target_dir": "/Users/rlx/datasets/google_maps/alcala_henares_20180211_L2_labels_train/",
           "overlay_original": False, "verbose": False},
         
         { "color_func": lambda x: func_label(x, alpha=.5), 
           "target_dir": "/Users/rlx/datasets/google_maps/alcala_henares_20180211_L2_overlay_train/",
           "overlay_original": True, "verbose": False, "default_alpha": 0.5}]


In [11]:
for img_fname in pbar()(imgs_train[3350:]):
    fname = confs[0]["target_dir"]+"/"+(".".join(img_fname.split(".")[:-1])+"_%s%s.jpg"%("L2", "")).split("/")[-1]
    if os.path.isfile(fname):
        continue
        
    k = geo.GoogleMaps_Static_Image.from_filename(img_fname)
    if k is None:
            continue
    L2.save_layer_patches_for_gmaps_img(k, confs)

100% (3650 of 3650) |#####################| Elapsed Time: 0:40:11 Time: 0:40:11
